# Summarizer v3

- Fine Tuning Base Model: Llama 3.2 (3B)
- 
Dataset: Dialogue Dataset from Kaggle

### Install

In [1]:
%%capture
!pip install pip3-autoremove
!pip-autoremove torch torchvision torchaudio -y
!pip install torch torchvision torchaudio xformers --index-url https://download.pytorch.org/whl/cu121
!pip install unsloth

### Import

In [ ]:
import os
os.environ["WANDB_PROJECT"] = "Summarizer_v2"
os.environ["WANDB_SILENT"] = "true"
os.environ["WANDB_API_KEY"] = ""

from unsloth import FastLanguageModel
import torch
import tqdm
from datasets import Dataset, load_dataset
import pandas as pd
from trl import SFTTrainer
from transformers import TrainingArguments, TextStreamer, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


### Load Model in 4 bit and Specify Parameters

In [3]:
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

==((====))==  Unsloth 2024.11.7: Fast Llama patching. Transformers = 4.46.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2024.11.7 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


### Load Dataset and Map it with the prompt

In [4]:
output_dir = "/kaggle/working/"
train_data_path = '/kaggle/input/summarization/dialogue_summary_01/dialogue_summary_train.csv'
val_data_path = '/kaggle/input/summarization/dialogue_summary_01/dialogue_summary_validation.csv'
train_df = pd.read_csv(train_data_path)
val_df = pd.read_csv(val_data_path)

In [5]:
train_df.head()

,dialogue,summary,topic
0,"#Person1#: Hi, Mr. Smith. I'm Doctor Hawkins. ...","Mr. Smith's getting a check-up, and Doctor Haw...",get a check-up
1,"#Person1#: Hello Mrs. Parker, how have you bee...",Mrs Parker takes Ricky for his vaccines. Dr. P...,vaccines
2,"#Person1#: Excuse me, did you see a set of key...",#Person1#'s looking for a set of keys and asks...,find keys
3,#Person1#: Why didn't you tell me you had a gi...,#Person1#'s angry because #Person2# didn't tel...,have a girlfriend
4,"#Person1#: Watsup, ladies! Y'll looking'fine t...",Malik invites Nikki to dance. Nikki agrees if ...,dance


In [6]:
val_df.head()

,dialogue,summary,topic
0,"#Person1#: Hello, how are you doing today?\n#P...",#Person2# has trouble breathing. The doctor as...,see a doctor
1,#Person1#: Hey Jimmy. Let's go workout later t...,#Person1# invites Jimmy to go workout and pers...,do exercise
2,#Person1#: I need to stop eating such unhealth...,#Person1# plans to stop eating unhealthy foods...,healthy foods
3,#Person1#: Do you believe in UFOs?\n#Person2#:...,#Person2# believes in UFOs and can see them in...,UFOs and aliens
4,#Person1#: Did you go to school today?\n#Perso...,#Person1# didn't go to school today. #Person2#...,go to school


In [7]:
prompt = """Below is a dialogue. Generate only an abstractive summary and a topic for the dialogue (without including the dialogue in the response).

### Dialogue:
{}

### Summary and Topic:
Summary: {}
Topic: {}
"""

EOS_TOKEN = tokenizer.eos_token  # Must add EOS_TOKEN

def formatting_prompts_func(examples):
    dialogues = examples["dialogue"]
    summaries = examples["summary"]
    topics = examples["topic"]
    
    texts = []
    for dialogue, summary, topic in zip(dialogues, summaries, topics):
        # Construct the prompt for training
        input_text = dialogue
        output_text = f"Summary: {summary} Topic: {topic}"
        
        # Ensure that EOS_TOKEN is added to avoid infinite generation
        text = prompt.format(input_text, summary, topic) + EOS_TOKEN
        texts.append(text)
    
    return {"text": texts}


# Convert the pandas DataFrame to the Hugging Face dataset
train_dataset = Dataset.from_pandas(train_df)
train_dataset = train_dataset.map(formatting_prompts_func, batched=True)

# Apply the same transformation to the validation dataset
val_dataset = Dataset.from_pandas(val_df)
val_dataset = val_dataset.map(formatting_prompts_func, batched=True)

Map:   0%|          | 0/12460 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [8]:
print(train_dataset[0])
# print(val_dataset[0]) 

{'dialogue': "#Person1#: Hi, Mr. Smith. I'm Doctor Hawkins. Why are you here today?\n#Person2#: I found it would be a good idea to get a check-up.\n#Person1#: Yes, well, you haven't had one for 5 years. You should have one every year.\n#Person2#: I know. I figure as long as there is nothing wrong, why go see the doctor?\n#Person1#: Well, the best way to avoid serious illnesses is to find out about them early. So try to come at least once a year for your own good.\n#Person2#: Ok.\n#Person1#: Let me see here. Your eyes and ears look fine. Take a deep breath, please. Do you smoke, Mr. Smith?\n#Person2#: Yes.\n#Person1#: Smoking is the leading cause of lung cancer and heart disease, you know. You really should quit.\n#Person2#: I've tried hundreds of times, but I just can't seem to kick the habit.\n#Person1#: Well, we have classes and some medications that might help. I'll give you more information before you leave.\n#Person2#: Ok, thanks doctor.", 'summary': "Mr. Smith's getting a check-u

In [9]:
print(train_dataset['text'][0])
# print(val_dataset['text'][0]) 

Below is a dialogue. Generate only an abstractive summary and a topic for the dialogue (without including the dialogue in the response).

### Dialogue:
#Person1#: Hi, Mr. Smith. I'm Doctor Hawkins. Why are you here today?
#Person2#: I found it would be a good idea to get a check-up.
#Person1#: Yes, well, you haven't had one for 5 years. You should have one every year.
#Person2#: I know. I figure as long as there is nothing wrong, why go see the doctor?
#Person1#: Well, the best way to avoid serious illnesses is to find out about them early. So try to come at least once a year for your own good.
#Person2#: Ok.
#Person1#: Let me see here. Your eyes and ears look fine. Take a deep breath, please. Do you smoke, Mr. Smith?
#Person2#: Yes.
#Person1#: Smoking is the leading cause of lung cancer and heart disease, you know. You really should quit.
#Person2#: I've tried hundreds of times, but I just can't seem to kick the habit.
#Person1#: Well, we have classes and some medications that might h

### Train

In [10]:
exist_already = os.path.exists(output_dir)

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    eval_dataset = val_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    packing = True,
    args = TrainingArguments(
        per_device_train_batch_size = 4,
        gradient_accumulation_steps = 4,
        warmup_steps = 10,
        num_train_epochs = 1,
        # max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 10,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = output_dir,
        save_strategy = "steps",
        save_steps = 5,
        eval_steps= 10,
        save_total_limit = 2,
        report_to="wandb"
    ),
)

In [ ]:
if(exist_already):
    trainer_stats = trainer.train(resume_from_checkpoint=True)
else:
    trainer_stats = trainer.train()

In [12]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 1,655 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 4 | Gradient Accumulation steps = 4
\        /    Total batch size = 16 | Total steps = 103
 "-____-"     Number of trainable parameters = 24,313,856
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss
10,1.710100
20,1.555300
30,1.464900
40,1.423200
50,1.403600
60,1.402900
70,1.416500
80,1.385800
90,1.392400
100,1.375600


### Save Model

In [13]:
model.save_pretrained("/kaggle/working/llama_3_2_3B_summarizer_v3")
tokenizer.save_pretrained("/kaggle/working/llama_3_2_3B_summarizer_v3")

('/kaggle/working/llama_3_2_3B_summarizer_v3/tokenizer_config.json',
 '/kaggle/working/llama_3_2_3B_summarizer_v3/special_tokens_map.json',
 '/kaggle/working/llama_3_2_3B_summarizer_v3/tokenizer.json')

### Inference

In [15]:
# Load the model and prepare it for inference (only done once)
model = FastLanguageModel.for_inference(model)

def generate_summary(user_input, max_new_tokens=256):
    # Preprocess the input to fit the prompt format
    summarization_prompt = """Below is a dialogue. Generate only an abstractive summary and a topic for the dialogue (without including the dialogue in the response).

        ### Dialogue:
        {}
        
        ### Summary and Topic:
        """
    inputs = tokenizer(
        [
            summarization_prompt.format(user_input)  # Insert the user input (article) into the prompt
        ], return_tensors="pt"
    ).to("cuda")  # Move inputs to GPU if available

    # Initialize the text streamer to handle model output
    text_streamer = TextStreamer(tokenizer)

    # Generate the summary
    _ = model.generate(
        **inputs,
        streamer=text_streamer,
        max_new_tokens=max_new_tokens,
        use_cache=True,
        temperature=0.4,  # randomness
        top_p=0.9,  # nucleus sampling for better coherence
    )

In [16]:
# Test the function
user_input = """
#Deepak#: "I've been feeling so sad and overwhelmed lately. Work has become such a massive source of stress for me."
#SurvUday#: "Hey there, I'm here to listen and support you. It sounds like work has been really challenging lately. Can you tell me more about what's been going on?"
#Deepak#: "I recently got a promotion at work, which I thought would be exciting. But the added responsibilities and pressure have just taken a toll on my mental health. It's been a really moving experience for me."
#SurvUday#: "I can understand how it can be overwhelming when we're faced with higher expectations. It's okay to acknowledge your emotions and allow yourself to feel sad in this situation. It's an important part of the healing process. What specific challenges have you been facing at work?"
#Deepak#: "Well, the workload has increased significantly, and I find it hard to maintain a work-life balance. I've been staying late at the office, and it feels like I'm constantly under a pile of never-ending tasks. It's just so hard to keep up, and it's impacting my overall well-being."
#SurvUday#: "It sounds like you're dealing with a lot of pressure to perform and succeed. Remember, it's crucial to take care of yourself, both mentally and physically. A healthy work-life balance is essential. Have you tried any strategies to cope with the added workload?"
#Deepak#: "I've been trying to prioritize my tasks and delegate whenever possible. I've also started practicing meditation during my breaks to help manage stress. But sometimes, it feels like no matter what I do, I can't catch a break. It's been a constant struggle."
#SurvUday#: "It's great to hear that you're already implementing some helpful strategies. Remember, progress takes time, and it's okay to have setbacks. In addition to what you're already doing, I encourage you to also communicate with your supervisor or team about your workload and discuss possible solutions together."
#Deepak#: "You're right. I haven't really opened up about my struggles to my coworkers or supervisor. I guess I've been afraid of appearing weak or incapable. How can I approach this discussion without feeling vulnerable?"
#SurvUday#: "It's completely normal to feel that way, but remember, asking for support is a strength, not a weakness. Start by scheduling a conversation with your supervisor or a trusted colleague in a private and comfortable setting. Be honest about your challenges and express your willingness to find solutions together. Remember, you're not alone in this."
#Deepak#: "Thank you for your understanding and guidance. I appreciate the reminder that I don't have to face this alone. I'll gather my courage and initiate that conversation soon. I need to prioritize my well-being and find a healthier balance."
#SurvUday#: "You're very welcome! I'm here to support you every step of the way. Taking care of yourself should always be a priority. Remember to be kind to yourself and celebrate your progress, no matter how small it may seem. You've got this!"
"""

# Call the function to generate the summary and topic
print(generate_summary(user_input))

<|begin_of_text|>Below is a dialogue. Generate only an abstractive summary and a topic for the dialogue (without including the dialogue in the response).

        ### Dialogue:
        
#Deepak#: "I've been feeling so sad and overwhelmed lately. Work has become such a massive source of stress for me."
#SurvUday#: "Hey there, I'm here to listen and support you. It sounds like work has been really challenging lately. Can you tell me more about what's been going on?"
#Deepak#: "I recently got a promotion at work, which I thought would be exciting. But the added responsibilities and pressure have just taken a toll on my mental health. It's been a really moving experience for me."
#SurvUday#: "I can understand how it can be overwhelming when we're faced with higher expectations. It's okay to acknowledge your emotions and allow yourself to feel sad in this situation. It's an important part of the healing process. What specific challenges have you been facing at work?"
#Deepak#: "Well, the wo

### Push Model to Huggingface

In [ ]:
Token = "" # Huggingface Token
Repo = "dkp2701/summarizer_v3" # Model Repo
model.push_to_hub_gguf(Repo,
                        tokenizer,
                        quantization_method = ["q4_k_m"],
                        token = Token)